In [148]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pickle
from sklearn.pipeline import make_pipeline
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score,mean_squared_error
from sklearn.manifold import TSNE

import os,cv2,math,glob,random
from math import cos, sin


In [86]:
# connecting to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
# Reading the whole data
df_raw = pd.read_csv('/content/drive/MyDrive/AFLW2000/data_normalized2.csv')
df_raw

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.001505,0.004881,-0.106576,-0.988322,-0.439510,0.440968,-1.057691,-0.010928,0.445039,-0.039062,...,-0.429542,0.180328,0.429219,0.239775,-0.434106,0.228478,-0.329915,0.027256,-0.247779,-0.010263
1,0.003351,0.003668,0.036870,-0.997498,0.207849,0.308620,0.289873,-0.286056,1.404030,-0.365910,...,-0.500342,0.564425,0.315701,0.502556,-0.542779,0.481239,-0.466242,-0.446169,0.684181,-0.270866
2,0.006657,0.006530,0.090390,-0.990206,-0.346265,0.593849,-0.968825,0.208624,1.262191,0.227498,...,-0.339541,0.526410,0.570050,0.615224,-0.309567,0.595393,-0.229283,0.093373,0.062462,-0.125822
3,0.001940,0.006611,-0.091248,-0.993814,-0.133685,0.345458,-0.497618,-0.086851,0.831509,-0.233359,...,-0.459218,0.359846,0.316974,0.357743,-0.545689,0.348940,-0.419761,-0.241359,0.205732,-0.150232
4,0.002696,0.007060,-0.113860,-0.987623,-0.389033,0.380386,-1.148294,-0.102541,0.190155,-0.144696,...,-0.482279,0.070495,0.345654,0.134283,-0.483069,0.118915,-0.368050,-0.167856,-0.361028,0.058591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,0.002330,0.001066,-0.052123,-0.992042,-0.295619,0.256864,-0.739186,-0.121943,0.729589,-0.182279,...,-0.464077,0.291408,0.251766,0.352682,-0.507572,0.333016,-0.396811,-0.199835,-0.007556,-0.035916
1849,0.004354,0.005374,0.669402,-0.742626,-0.472708,-0.039697,-0.575264,-0.920736,0.422518,0.029066,...,-0.715321,-0.131350,0.250336,0.526651,-0.212482,0.432978,-0.161053,-0.304740,0.074078,0.735418
1850,0.002207,0.007331,0.093183,-0.995112,-0.400055,0.333543,-0.989650,-0.186692,0.485125,0.020536,...,-0.488566,0.106907,0.388059,0.346836,-0.362784,0.321121,-0.271144,-0.329202,-0.237305,0.184649
1851,0.008220,0.006576,0.070039,-0.990451,-0.268616,0.459611,-0.742969,0.063757,0.936078,0.116944,...,-0.405234,0.357422,0.475522,0.506669,-0.374943,0.488737,-0.260024,-0.019941,0.076776,0.011841


In [30]:
df_raw.corr()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.000000,0.127860,-0.036173,0.035663,-0.010250,0.232248,-0.045115,0.182709,0.013764,0.118936,...,0.147063,0.029870,0.186858,-0.011880,0.080502,-0.000920,0.068608,-0.021665,0.036029,-0.025496
1,0.127860,1.000000,0.010630,-0.001514,-0.045623,0.229652,-0.044254,0.151917,0.019608,0.161856,...,0.101265,0.006811,0.236067,0.014481,0.112099,0.019271,0.109577,0.023161,0.012968,-0.015702
2,-0.036173,0.010630,1.000000,-0.060511,0.101921,-0.322338,0.485763,-0.498573,0.485845,0.526956,...,-0.535888,0.089845,0.370024,0.778678,0.536356,0.718620,0.543379,0.001292,-0.049975,0.023487
3,0.035663,-0.001514,-0.060511,1.000000,0.137545,-0.085244,0.120825,0.207989,-0.129192,0.116181,...,0.508530,-0.100123,-0.152086,-0.180528,0.421921,-0.171526,0.364631,0.008722,-0.004763,0.014690
4,-0.010250,-0.045623,0.101921,0.137545,1.000000,0.007750,0.901898,0.138041,0.742654,-0.361509,...,0.294338,0.791743,-0.261514,0.486446,-0.356029,0.538234,-0.358667,0.016448,0.042042,0.056912
5,0.232248,0.229652,-0.322338,-0.085244,0.007750,1.000000,-0.182413,0.864534,0.135443,0.358895,...,0.639583,0.238359,0.649624,-0.024380,0.092619,0.038647,0.127050,0.000738,0.028485,-0.002422
6,-0.045115,-0.044254,0.485763,0.120825,0.901898,-0.182413,1.000000,-0.098149,0.843802,-0.134574,...,0.045467,0.734994,-0.146436,0.733052,-0.113545,0.753971,-0.115392,0.007409,0.013734,0.059437
7,0.182709,0.151917,-0.498573,0.207989,0.138041,0.864534,-0.098149,1.000000,0.136538,0.195446,...,0.900657,0.358563,0.329166,-0.116395,0.020442,-0.036457,0.043759,-0.006177,0.042184,0.009496
8,0.013764,0.019608,0.485845,-0.129192,0.742654,0.135443,0.843802,0.136538,1.000000,0.100227,...,0.106423,0.897073,0.166276,0.908475,-0.038391,0.942312,0.000079,0.000590,0.018376,0.062083
9,0.118936,0.161856,0.526956,0.116181,-0.361509,0.358895,-0.134574,0.195446,0.100227,1.000000,...,-0.019318,-0.151453,0.869418,0.346465,0.903565,0.312823,0.930161,0.001383,-0.051268,0.013255


# The order of columns is X - Y - Pitch - Yaw - Roll 


In [88]:
# Sparating features from labels
features = df_raw.iloc[:,:22]
features

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.001505,0.004881,-0.106576,-0.988322,-0.439510,0.440968,-1.057691,-0.010928,0.445039,-0.039062,...,-0.118111,0.883197,-0.526940,-0.429542,0.180328,0.429219,0.239775,-0.434106,0.228478,-0.329915
1,0.003351,0.003668,0.036870,-0.997498,0.207849,0.308620,0.289873,-0.286056,1.404030,-0.365910,...,0.514540,0.711558,0.017889,-0.500342,0.564425,0.315701,0.502556,-0.542779,0.481239,-0.466242
2,0.006657,0.006530,0.090390,-0.990206,-0.346265,0.593849,-0.968825,0.208624,1.262191,0.227498,...,0.064149,1.276109,-0.442559,-0.339541,0.526410,0.570050,0.615224,-0.309567,0.595393,-0.229283
3,0.001940,0.006611,-0.091248,-0.993814,-0.133685,0.345458,-0.497618,-0.086851,0.831509,-0.233359,...,0.181704,0.702734,-0.327107,-0.459218,0.359846,0.316974,0.357743,-0.545689,0.348940,-0.419761
4,0.002696,0.007060,-0.113860,-0.987623,-0.389033,0.380386,-1.148294,-0.102541,0.190155,-0.144696,...,-0.189495,0.742289,-0.522197,-0.482279,0.070495,0.345654,0.134283,-0.483069,0.118915,-0.368050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,0.002330,0.001066,-0.052123,-0.992042,-0.295619,0.256864,-0.739186,-0.121943,0.729589,-0.182279,...,0.031116,0.690248,-0.385512,-0.464077,0.291408,0.251766,0.352682,-0.507572,0.333016,-0.396811
1849,0.004354,0.005374,0.669402,-0.742626,-0.472708,-0.039697,-0.575264,-0.920736,0.422518,0.029066,...,-0.589710,0.330579,0.013263,-0.715321,-0.131350,0.250336,0.526651,-0.212482,0.432978,-0.161053
1850,0.002207,0.007331,0.093183,-0.995112,-0.400055,0.333543,-0.989650,-0.186692,0.485125,0.020536,...,-0.232949,0.784064,-0.434114,-0.488566,0.106907,0.388059,0.346836,-0.362784,0.321121,-0.271144
1851,0.008220,0.006576,0.070039,-0.990451,-0.268616,0.459611,-0.742969,0.063757,0.936078,0.116944,...,0.021728,0.982702,-0.378793,-0.405234,0.357422,0.475522,0.506669,-0.374943,0.488737,-0.260024


In [89]:
# Extracting
labels = df_raw.iloc[:,22:]
pitch_raw = labels.iloc[:,0]
yaw_raw = labels.iloc[:,1]
roll_raw = labels.iloc[:,2]

In [90]:
labels.describe()

,22,23,24
count,1853.000000,1853.000000,1853.000000
mean,-0.095366,0.013611,-0.033167
std,0.549292,0.580266,0.649282
min,-12.457202,-6.130066,-16.969362
25%,-0.246453,-0.256714,-0.160274
50%,-0.123371,0.019403,-0.017463
75%,0.028133,0.305791,0.126460
max,15.598820,3.277500,2.166107


In [103]:
# Spliting data 
x_train, x_test, y_train_pitch, y_test_pitch = train_test_split(features, pitch_raw, test_size=0.2,shuffle=True ,random_state=1234)
x_train, x_test, y_train_yaw, y_test_yaw = train_test_split(features, yaw_raw, test_size=0.2, shuffle=True, random_state=1234)
x_train, x_test, y_train_roll, y_test_roll = train_test_split(features, roll_raw, test_size=0.2,shuffle=True, random_state=1234)


In [134]:
model_pitch = svm.SVR().fit(x_train,y_train_pitch)
model_yaw = RandomForestRegressor().fit(x_train,y_train_yaw)


In [144]:
pipe = make_pipeline(PCA(),RandomForestRegressor())
model_roll = pipe.fit(x_train,y_train_roll)

In [145]:
print(f'Pitch SCORES are:')
print(f'Mean Square Error is{mean_squared_error(y_train_pitch,model_pitch.predict(x_train))}')
print(f'R2 Score is: {r2_score(y_test_pitch,model_pitch.predict(x_test))}')

print('=========================================')

print(f'Yaw SCORES are:')
print(f'Mean Square Error is{mean_squared_error(y_train_yaw,model_yaw.predict(x_train))}')
print(f'R2 Score is: {r2_score(y_test_yaw,model_yaw.predict(x_test))}')

print('===========================================')

print(f'Roll SCORES are:')
print(f'Mean Square Error is{mean_squared_error(y_train_roll,model_roll.predict(x_train))}')
print(f'R2 Score is: {r2_score(y_test_roll,model_roll.predict(x_test))}')




Pitch SCORES are:
Mean Square Error is0.29040406931213436
R2 Score is: 0.5084983015097
Yaw SCORES are:
Mean Square Error is0.007610838292824154
R2 Score is: 0.7735094443702225
Roll SCORES are:
Mean Square Error is0.05483425347027014
R2 Score is: -2.7716712431484516
